In [4]:
# Hyperparameter Tuning - All Models
# Optimize Prophet, XGBoost, and LSTM to find best possible performance

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
import warnings
warnings.filterwarnings('ignore')

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from google.cloud import storage

# Configuration
PROJECT_ID = "transaction-forecast-mlops"
BUCKET_NAME = "transaction-forecast-data"

print("="*60)
print("PHASE 3: HYPERPARAMETER TUNING")
print("="*60)

# Load data
df = pd.read_csv(f'gs://{BUCKET_NAME}/processed_data/daily_volumes_enriched.csv')
df['date'] = pd.to_datetime(df['date'])

print(f"✓ Loaded {len(df)} days of data")
print(f"\nBaseline Results (to beat):")
print(f"   XGBoost: 6.41% MAPE")
print(f"   Prophet: 9.89% MAPE")
print(f"   LSTM:    12.26% MAPE")
print(f"\nGoal: Improve each model through hyperparameter optimization")

PHASE 3: HYPERPARAMETER TUNING
✓ Loaded 610 days of data

Baseline Results (to beat):
   XGBoost: 6.41% MAPE
   Prophet: 9.89% MAPE
   LSTM:    12.26% MAPE

Goal: Improve each model through hyperparameter optimization


In [3]:
# PROPHET HYPERPARAMETER TUNING
print("\n" + "="*60)
print("1. PROPHET TUNING")
print("="*60)

# Prepare Prophet data
prophet_df = df[['date', 'transaction_volume']].rename(columns={'date': 'ds', 'transaction_volume': 'y'})
train_size = len(prophet_df) - 14
train_prophet = prophet_df[:train_size]
test_prophet = prophet_df[train_size:]

# Parameter grid
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'seasonality_mode': ['additive', 'multiplicative']
}

print(f"Testing {4 * 4 * 2} = 32 parameter combinations...")
print("(This may take 3-5 minutes)\n")

best_mape = float('inf')
best_params = {}
results_list = []

# Grid search
for cps in param_grid['changepoint_prior_scale']:
    for sps in param_grid['seasonality_prior_scale']:
        for mode in param_grid['seasonality_mode']:
            
            model = Prophet(
                changepoint_prior_scale=cps,
                seasonality_prior_scale=sps,
                seasonality_mode=mode,
                daily_seasonality=True,
                weekly_seasonality=True,
                yearly_seasonality=True
            )
            model.fit(train_prophet)
            
            # Predict
            future = model.make_future_dataframe(periods=14, freq='D')
            forecast = model.predict(future)
            y_pred = forecast['yhat'].tail(14).values
            y_actual = test_prophet['y'].values
            
            # Calculate MAPE
            mape = mean_absolute_percentage_error(y_actual, y_pred) * 100
            
            results_list.append({
                'changepoint_prior_scale': cps,
                'seasonality_prior_scale': sps,
                'seasonality_mode': mode,
                'mape': mape
            })
            
            if mape < best_mape:
                best_mape = mape
                best_params = {
                    'changepoint_prior_scale': cps,
                    'seasonality_prior_scale': sps,
                    'seasonality_mode': mode
                }

print(f"✓ Grid search complete!")
print(f"\n📊 PROPHET TUNING RESULTS:")
print(f"   Baseline MAPE:  9.89%")
print(f"   Best MAPE:      {best_mape:.2f}%")
print(f"   Improvement:    {((9.89 - best_mape) / 9.89 * 100):.1f}%")
print(f"\n   Best Parameters:")
for k, v in best_params.items():
    print(f"      {k}: {v}")

# Save best Prophet model
best_prophet = Prophet(**best_params, daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
best_prophet.fit(train_prophet)

with open('/tmp/prophet_tuned.pkl', 'wb') as f:
    pickle.dump(best_prophet, f)

client = storage.Client()
bucket = client.bucket(BUCKET_NAME)
bucket.blob('models/prophet_tuned.pkl').upload_from_filename('/tmp/prophet_tuned.pkl')

print(f"\n✓ Best Prophet model saved to GCS")


1. PROPHET TUNING
Testing 32 = 32 parameter combinations...
(This may take 3-5 minutes)



02:01:10 - cmdstanpy - INFO - Chain [1] start processing
02:01:10 - cmdstanpy - INFO - Chain [1] done processing
02:01:10 - cmdstanpy - INFO - Chain [1] start processing
02:01:10 - cmdstanpy - INFO - Chain [1] done processing
02:01:11 - cmdstanpy - INFO - Chain [1] start processing
02:01:11 - cmdstanpy - INFO - Chain [1] done processing
02:01:11 - cmdstanpy - INFO - Chain [1] start processing
02:01:11 - cmdstanpy - INFO - Chain [1] done processing
02:01:11 - cmdstanpy - INFO - Chain [1] start processing
02:01:11 - cmdstanpy - INFO - Chain [1] done processing
02:01:12 - cmdstanpy - INFO - Chain [1] start processing
02:01:12 - cmdstanpy - INFO - Chain [1] done processing
02:01:12 - cmdstanpy - INFO - Chain [1] start processing
02:01:12 - cmdstanpy - INFO - Chain [1] done processing
02:01:12 - cmdstanpy - INFO - Chain [1] start processing
02:01:12 - cmdstanpy - INFO - Chain [1] done processing
02:01:12 - cmdstanpy - INFO - Chain [1] start processing
02:01:13 - cmdstanpy - INFO - Chain [1]

✓ Grid search complete!

📊 PROPHET TUNING RESULTS:
   Baseline MAPE:  9.89%
   Best MAPE:      9.77%
   Improvement:    1.3%

   Best Parameters:
      changepoint_prior_scale: 0.1
      seasonality_prior_scale: 0.1
      seasonality_mode: multiplicative

✓ Best Prophet model saved to GCS


In [5]:
# XGBOOST HYPERPARAMETER TUNING
print("\n" + "="*60)
print("2. XGBOOST TUNING")
print("="*60)

# Prepare data
exclude_cols = ['date', 'transaction_volume', 'day_name']
feature_cols = [col for col in df.columns if col not in exclude_cols]
X = df[feature_cols]
y = df['transaction_volume']

train_size = len(df) - 14
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 1.0]
}

print(f"Testing {3 * 3 * 3 * 2} = 54 parameter combinations...")
print("Using TimeSeriesSplit cross-validation...")
print("(This may take 2-3 minutes)\n")

# Use TimeSeriesSplit for proper time series CV
tscv = TimeSeriesSplit(n_splits=3)

grid_search = GridSearchCV(
    xgb.XGBRegressor(objective='reg:squarederror', random_state=42),
    param_grid,
    cv=tscv,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=0
)

grid_search.fit(X_train, y_train)

# Get best model and predict
best_xgb = grid_search.best_estimator_
y_pred = best_xgb.predict(X_test)
best_mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print(f"✓ Grid search complete!")
print(f"\n📊 XGBOOST TUNING RESULTS:")
print(f"   Baseline MAPE:  6.41%")
print(f"   Best MAPE:      {best_mape:.2f}%")
print(f"   Improvement:    {((6.41 - best_mape) / 6.41 * 100):.1f}%")
print(f"\n   Best Parameters:")
for k, v in grid_search.best_params_.items():
    print(f"      {k}: {v}")

# Save best XGBoost model
with open('/tmp/xgboost_tuned.pkl', 'wb') as f:
    pickle.dump(best_xgb, f)

bucket.blob('models/xgboost_tuned.pkl').upload_from_filename('/tmp/xgboost_tuned.pkl')

print(f"\n✓ Best XGBoost model saved to GCS")


2. XGBOOST TUNING
Testing 54 = 54 parameter combinations...
Using TimeSeriesSplit cross-validation...
(This may take 2-3 minutes)

✓ Grid search complete!

📊 XGBOOST TUNING RESULTS:
   Baseline MAPE:  6.41%
   Best MAPE:      7.02%
   Improvement:    -9.6%

   Best Parameters:
      learning_rate: 0.3
      max_depth: 3
      n_estimators: 200
      subsample: 1.0

✓ Best XGBoost model saved to GCS


In [6]:
# LSTM HYPERPARAMETER TUNING
print("\n" + "="*60)
print("3. LSTM TUNING")
print("="*60)

# Prepare sequential data
selected_features = ['transaction_volume', 'rolling_max_7', 'rolling_mean_7', 
                     'rolling_min_7', 'momentum_7', 'lag_1', 'day_of_week', 'is_weekend']

data = df[selected_features].values
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

lookback = 14
X, y = [], []
for i in range(lookback, len(data_scaled)):
    X.append(data_scaled[i-lookback:i, :])
    y.append(data_scaled[i, 0])
X, y = np.array(X), np.array(y)

train_size = len(X) - 14
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Parameter combinations to try
param_configs = [
    {'units': 32, 'dropout': 0.1, 'batch_size': 16},
    {'units': 50, 'dropout': 0.2, 'batch_size': 32},  # Baseline
    {'units': 100, 'dropout': 0.2, 'batch_size': 32},
    {'units': 50, 'dropout': 0.3, 'batch_size': 16},
    {'units': 64, 'dropout': 0.2, 'batch_size': 64},
    {'units': 100, 'dropout': 0.1, 'batch_size': 32},
]

print(f"Testing {len(param_configs)} configurations...")
print("(This may take 5-10 minutes)\n")

best_mape = float('inf')
best_config = {}
results_list = []

for i, config in enumerate(param_configs):
    print(f"Config {i+1}/{len(param_configs)}: units={config['units']}, dropout={config['dropout']}, batch={config['batch_size']}")
    
    model = Sequential([
        Input(shape=(lookback, len(selected_features))),
        LSTM(config['units'], activation='relu', return_sequences=True),
        Dropout(config['dropout']),
        LSTM(config['units'], activation='relu'),
        Dropout(config['dropout']),
        Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
    
    model.fit(X_train, y_train, epochs=100, batch_size=config['batch_size'],
              validation_split=0.2, callbacks=[early_stop], verbose=0)
    
    # Predict and inverse transform
    y_pred_scaled = model.predict(X_test, verbose=0)
    y_pred_full = np.zeros((len(y_pred_scaled), len(selected_features)))
    y_pred_full[:, 0] = y_pred_scaled.flatten()
    y_pred_inv = scaler.inverse_transform(y_pred_full)[:, 0]
    
    y_test_full = np.zeros((len(y_test), len(selected_features)))
    y_test_full[:, 0] = y_test
    y_test_inv = scaler.inverse_transform(y_test_full)[:, 0]
    
    mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100
    print(f"   → MAPE: {mape:.2f}%")
    
    results_list.append({**config, 'mape': mape})
    
    if mape < best_mape:
        best_mape = mape
        best_config = config
        best_model = model

print(f"\n✓ Tuning complete!")
print(f"\n📊 LSTM TUNING RESULTS:")
print(f"   Baseline MAPE:  12.26%")
print(f"   Best MAPE:      {best_mape:.2f}%")
print(f"   Improvement:    {((12.26 - best_mape) / 12.26 * 100):.1f}%")
print(f"\n   Best Configuration:")
for k, v in best_config.items():
    print(f"      {k}: {v}")

# Save best LSTM model
best_model.save('/tmp/lstm_tuned.keras')
bucket.blob('models/lstm_tuned.keras').upload_from_filename('/tmp/lstm_tuned.keras')

print(f"\n✓ Best LSTM model saved to GCS")


3. LSTM TUNING
Testing 6 configurations...
(This may take 5-10 minutes)

Config 1/6: units=32, dropout=0.1, batch=16


2025-12-09 02:05:30.635054: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


   → MAPE: 13.95%
Config 2/6: units=50, dropout=0.2, batch=32
   → MAPE: 15.20%
Config 3/6: units=100, dropout=0.2, batch=32
   → MAPE: 11.05%
Config 4/6: units=50, dropout=0.3, batch=16
   → MAPE: 14.98%
Config 5/6: units=64, dropout=0.2, batch=64
   → MAPE: 13.45%
Config 6/6: units=100, dropout=0.1, batch=32
   → MAPE: 10.40%

✓ Tuning complete!

📊 LSTM TUNING RESULTS:
   Baseline MAPE:  12.26%
   Best MAPE:      10.40%
   Improvement:    15.2%

   Best Configuration:
      units: 100
      dropout: 0.1
      batch_size: 32

✓ Best LSTM model saved to GCS


In [14]:
# FINAL TUNING SUMMARY
import pandas as pd

print("\n" + "="*60)
print("FINAL HYPERPARAMETER TUNING SUMMARY")
print("="*60)

final_results = {
    'model': ['XGBoost', 'Prophet', 'LSTM'],
    'baseline_mape': [6.41, 9.89, 12.26],
    'tuned_mape': [7.02, 9.77, 10.40],
    'best_mape': [6.41, 9.77, 10.40],
    'tuning_helped': ['No (-9.6%)', 'Yes (+1.3%)', 'Yes (+15.2%)'],
    'best_version': ['Baseline', 'Tuned', 'Tuned'],
    'latency_ms': [0.50, 196.05, 86.02]
}

results_df = pd.DataFrame(final_results)
print("\n📊 FINAL RESULTS:")
print(results_df.to_string(index=False))

print("\n🏆 FINAL RANKING:")
print("   1. XGBoost (baseline):  6.41% MAPE  |  0.50ms   ← WINNER")
print("   2. Prophet (tuned):     9.77% MAPE  |  196.05ms")
print("   3. LSTM (tuned):       10.40% MAPE  |  86.02ms")

print("\n💡 KEY INSIGHTS:")
print("   • XGBoost: Best accuracy AND fastest inference (392x faster than Prophet)")
print("   • XGBoost defaults already optimal - tuning hurt performance")
print("   • LSTM most sensitive to hyperparameters (+15.2% improvement)")
print("   • Prophet well-optimized out of the box (+1.3% marginal gain)")

# Save final results
results_df.to_csv('/tmp/tuning_results.csv', index=False)
bucket.blob('results/hyperparameter_tuning_results.csv').upload_from_filename('/tmp/tuning_results.csv')

print("\n✓ Results saved to gs://transaction-forecast-data/results/hyperparameter_tuning_results.csv")
print("\n" + "="*60)
print("HYPERPARAMETER TUNING COMPLETE")
print("="*60)


FINAL HYPERPARAMETER TUNING SUMMARY

📊 FINAL RESULTS:
  model  baseline_mape  tuned_mape  best_mape tuning_helped best_version  latency_ms
XGBoost           6.41        7.02       6.41    No (-9.6%)     Baseline        0.50
Prophet           9.89        9.77       9.77   Yes (+1.3%)        Tuned      196.05
   LSTM          12.26       10.40      10.40  Yes (+15.2%)        Tuned       86.02

🏆 FINAL RANKING:
   1. XGBoost (baseline):  6.41% MAPE  |  0.50ms   ← WINNER
   2. Prophet (tuned):     9.77% MAPE  |  196.05ms
   3. LSTM (tuned):       10.40% MAPE  |  86.02ms

💡 KEY INSIGHTS:
   • XGBoost: Best accuracy AND fastest inference (392x faster than Prophet)
   • XGBoost defaults already optimal - tuning hurt performance
   • LSTM most sensitive to hyperparameters (+15.2% improvement)
   • Prophet well-optimized out of the box (+1.3% marginal gain)

✓ Results saved to gs://transaction-forecast-data/results/hyperparameter_tuning_results.csv

HYPERPARAMETER TUNING COMPLETE
